## Introduction: Business Problem

Yoga has seen a surge in popularity in **Singapore** in the past few years. In this project, we strive to recommend areas in **Singapore** to open a new yoga studio. 
The aim is to avoid opening the studio in areas with high competition. We will highlight urban clusters 
that have a high density of yoga studios to avoid.


### Install the necessary libararies

In [12]:
!pip install pandas==1.0.3

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation



!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 10.2 MB 11.1 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5
Solving environment: done

## Package Plan ##

  environment location: /Users/eugenetay/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.4                |   py38h32f6830_2         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy        

### Foursquare Data

Set credentials for authentication and set location to Singapore to obtain data of all Yoga studios within a 27km radius of the island state

In [13]:
CLIENT_ID = '5KH4LR10VGSOBKMAXPBL3K1JDHWYWTKCTLT0YDACAYSNEEIW' # your Foursquare ID
CLIENT_SECRET = 'UPBPCTJ3FLBXO1S5E1KAQ3SHQTTAIYBRCBGJBIFUY3DVOXD2' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: 5KH4LR10VGSOBKMAXPBL3K1JDHWYWTKCTLT0YDACAYSNEEIW
CLIENT_SECRET:UPBPCTJ3FLBXO1S5E1KAQ3SHQTTAIYBRCBGJBIFUY3DVOXD2


In [14]:
address = 'Singapore'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

1.3408630000000001 103.83039182212079


In [28]:
search_query = 'Yoga'
radius = 27000

In [29]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=5KH4LR10VGSOBKMAXPBL3K1JDHWYWTKCTLT0YDACAYSNEEIW&client_secret=UPBPCTJ3FLBXO1S5E1KAQ3SHQTTAIYBRCBGJBIFUY3DVOXD2&ll=1.3408630000000001,103.83039182212079&v=20180604&query=Yoga&radius=27000&limit=30'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f329a8c1cb0e54c1ec253d4'},
 'response': {'venues': [{'id': '4b8baf58f964a520f1a632e3',
    'name': 'True Yoga',
    'location': {'address': '#04-01, Pacific Plaza',
     'crossStreet': '9 Scotts Rd',
     'lat': 1.306395185101882,
     'lng': 103.83222149257372,
     'labeledLatLngs': [{'label': 'display',
       'lat': 1.306395185101882,
       'lng': 103.83222149257372}],
     'distance': 3842,
     'postalCode': '228210',
     'cc': 'SG',
     'city': 'Singapore',
     'country': 'Singapore',
     'formattedAddress': ['#04-01, Pacific Plaza (9 Scotts Rd)',
      '228210',
      'Singapore']},
    'categories': [{'id': '4bf58dd8d48988d102941735',
      'name': 'Yoga Studio',
      'pluralName': 'Yoga Studios',
      'shortName': 'Yoga Studio',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1597151999',
    'hasPerk': False},
   {'id':

In [32]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.country,location.formattedAddress,location.neighborhood,venuePage.id,location.state
0,4b8baf58f964a520f1a632e3,True Yoga,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",v-1597151999,False,"#04-01, Pacific Plaza",9 Scotts Rd,1.306395,103.832221,"[{'label': 'display', 'lat': 1.306395185101882...",3842,228210,SG,Singapore,Singapore,"[#04-01, Pacific Plaza (9 Scotts Rd), 228210, ...",NaN,NaN,NaN
1,4d5a6c03e092a35d456e0a0c,Yoga Studio,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",v-1597151999,False,53 Lorong 5 Toa Payoh,NaN,1.335600,103.850433,"[{'label': 'display', 'lat': 1.335600433070415...",2305,NaN,SG,Singapore,Singapore,"[53 Lorong 5 Toa Payoh, Singapore]",NaN,NaN,NaN
2,504f6afbe4b05f0233684f50,Hom Yoga,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",v-1597151999,False,"#06-12/13, Orchard Central",181 Orchard Rd.,1.300824,103.839795,"[{'label': 'display', 'lat': 1.300824054472386...",4578,238896,SG,Singapore,Singapore,"[#06-12/13, Orchard Central (181 Orchard Rd.),...",NaN,NaN,NaN
3,53846159498e5da6508f19a5,Yoga Movement,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",v-1597151999,False,22 Orchard Rd,NaN,1.299164,103.846912,"[{'label': 'display', 'lat': 1.299164328163440...",4992,NaN,SG,NaN,Singapore,"[22 Orchard Rd, Singapore]",NaN,NaN,NaN
4,5b9ca90cefa82a002c0ed3dc,Yoga+,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",v-1597151999,False,89 Tanjong Pagar road,NaN,1.277620,103.843765,"[{'label': 'display', 'lat': 1.2776201, 'lng':...",7195,088512,SG,Singapore,Singapore,"[89 Tanjong Pagar road, 088512, Singapore]",NaN,NaN,NaN


Clean up the dataframe to show categories

In [33]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,country,formattedAddress,neighborhood,state,id
0,True Yoga,Yoga Studio,"#04-01, Pacific Plaza",9 Scotts Rd,1.306395,103.832221,"[{'label': 'display', 'lat': 1.306395185101882...",3842,228210,SG,Singapore,Singapore,"[#04-01, Pacific Plaza (9 Scotts Rd), 228210, ...",NaN,NaN,4b8baf58f964a520f1a632e3
1,Yoga Studio,Yoga Studio,53 Lorong 5 Toa Payoh,NaN,1.335600,103.850433,"[{'label': 'display', 'lat': 1.335600433070415...",2305,NaN,SG,Singapore,Singapore,"[53 Lorong 5 Toa Payoh, Singapore]",NaN,NaN,4d5a6c03e092a35d456e0a0c
2,Hom Yoga,Yoga Studio,"#06-12/13, Orchard Central",181 Orchard Rd.,1.300824,103.839795,"[{'label': 'display', 'lat': 1.300824054472386...",4578,238896,SG,Singapore,Singapore,"[#06-12/13, Orchard Central (181 Orchard Rd.),...",NaN,NaN,504f6afbe4b05f0233684f50
3,Yoga Movement,Yoga Studio,22 Orchard Rd,NaN,1.299164,103.846912,"[{'label': 'display', 'lat': 1.299164328163440...",4992,NaN,SG,NaN,Singapore,"[22 Orchard Rd, Singapore]",NaN,NaN,53846159498e5da6508f19a5
4,Yoga+,Yoga Studio,89 Tanjong Pagar road,NaN,1.277620,103.843765,"[{'label': 'display', 'lat': 1.2776201, 'lng':...",7195,088512,SG,Singapore,Singapore,"[89 Tanjong Pagar road, 088512, Singapore]",NaN,NaN,5b9ca90cefa82a002c0ed3dc
5,Pure Yoga,Yoga Studio,"#18-00, Ngee Ann City Tower A",391A Orchard Rd,1.302337,103.835082,"[{'label': 'display', 'lat': 1.302337394876801...",4320,238873,SG,Singapore,Singapore,"[#18-00, Ngee Ann City Tower A (391A Orchard R...",NaN,NaN,4b4afd63f964a5209b9026e3
6,Bikram Original Hot Yoga,Yoga Studio,#05-01 Pacific Plaza,9 Scotts Road,1.306453,103.832169,"[{'label': 'display', 'lat': 1.30645261006179,...",3835,228210,SG,Singapore,Singapore,"[#05-01 Pacific Plaza (9 Scotts Road), 228210,...",NaN,NaN,4b5eed95f964a520429e29e3
7,Yoga Movement,Yoga Studio,21 Tanjong Pagar Rd,Murray St,1.279561,103.844026,"[{'label': 'display', 'lat': 1.279561008004445...",6990,088444,SG,Singapore,Singapore,"[21 Tanjong Pagar Rd (Murray St), 088444, Sing...",NaN,NaN,51a49cd7498e45605109bc28
8,Yoga Studio @ Thomson,Yoga Studio,NaN,NaN,1.341911,103.838153,"[{'label': 'display', 'lat': 1.341910523846336...",871,NaN,SG,NaN,Singapore,[Singapore],NaN,NaN,4fd5bdace4b07b066331fed4
9,Hom Yoga,Yoga Studio,"3 Canton St, #02-01",NaN,1.286237,103.849389,"[{'label': 'display', 'lat': 1.286236796897962...",6437,049745,SG,Singapore,Singapore,"[3 Canton St, #02-01, 049745, Singapore]",NaN,NaN,4c14a0457f7f2d7fd234e168
